# The Annotated Hellogame

### Disclaimer
This notebook closely follows the tutorial steps from **docs/the_annotated_hellogame.ipynb** in the [**clp-research/clembench**](https://github.com/clp-research/clembench) repository. <br> 
While the core content is derived from the original tutorial, some additional detailed descriptions and explanations have been added to serve as a source of knowledge during the development of our project.

### Purpose
This tutorial provides insight into how to add a game to the clembench distribution. <br> 
It uses all abstract classes, which is not a must, but handy if you would like to create output that is expected from the evaluating scripts.

#### In this Setup:
- Define in advance a number of game instances, like a target to guess in a guessing game.
- Single standardized entry point for running the game, logging the results, and producing specific scores per episode, that will ultimately enter into the overall benchmark.

In [1]:
import string
import sys
from typing import Dict, List

sys.path.append('/home/zoltan/Desktop/PROII/Gruppe-C-PRO2-Projekt/clembench-main')

from backends import Model, CustomResponseModel, ModelSpec, load_model_registry, get_model_for
from clemgame.clemgame import GameMaster, GameBenchmark, Player, DialogueGameMaster
from clemgame import get_logger


      _                _                     _     
     | |              | |                   | |    
  ___| | ___ _ __ ___ | |__   ___ _ __   ___| |__  
 / __| |/ _ \ '_ ` _ \| '_ \ / _ \ '_ \ / __| '_ \ 
| (__| |  __/ | | | | | |_) |  __/ | | | (__| | | |
 \___|_|\___|_| |_| |_|_.__/ \___|_| |_|\___|_| |_|

No module named 'imageio'


Cannot load 'games.mm_mapworld_specificroom.master'. Please make sure that the file exists.


No module named 'tiktoken'
No module named 'imageio'
No module named 'tiktoken'
No module named 'imageio'
No module named 'socketio'


Cannot load 'games.wordle_withcritic.master'. Please make sure that the file exists.
Cannot load 'games.mm_mapworld_qa.master'. Please make sure that the file exists.
Cannot load 'games.wordle.master'. Please make sure that the file exists.
Cannot load 'games.mm_mapworld_graphs.master'. Please make sure that the file exists.
Cannot load 'games.chatgame.master'. Please make sure that the file exists.


No module named 'tiktoken'
No module named 'imageio'


Cannot load 'games.wordle_withclue.master'. Please make sure that the file exists.
Cannot load 'games.mm_mapworld.master'. Please make sure that the file exists.


In [2]:
load_model_registry()
logger = get_logger(__name__)

In [3]:
GAME_NAME = "hellogame"

## HelloGame Class
The functionality of the class is pretty straight forward. How it is inherited or being used by our game is potentially up to the structure of how we want to implement the game. <br>
Cool thing: in the **_on_setup** method, game_instance is passed as an argument with two *s. <br> 
This basically means that the method can take additional keyword arguments. 

In [4]:
class HelloGame(DialogueGameMaster):
    """This class implements a greeting game in which player A
    is greeting another player with a target name.
    """

    def __init__(self, experiment: Dict, player_models: List[Model]):
        super().__init__(GAME_NAME, experiment, player_models)
        self.language: int = experiment["language"]  # fetch experiment parameters here
        self.turns = []
        self.required_words = ["welcome", "hello"]
        self.success = True

    def _on_setup(self, **game_instance):
        self.game_instance = game_instance  # fetch game parameters here

        # Create the players
        self.greeted = Greeted(game_instance["target_name"])
        self.greeter = Greeter(self.player_models[0])

        # Add the players: these will be logged to the records interactions.json
        # NOTE: During game play the players will be called in the order added here
        self.add_player(self.greeter)
        self.add_player(self.greeted)

        self.required_words.append(self.greeted.name.lower())

    def _on_before_game(self):
        # Do something before the game start e.g. add the initial prompts to the message list for the players
        self.add_user_message(self.greeter, self.game_instance["prompt"])

    def _does_game_proceed(self):
        # Determine if the game should proceed. This is also called once initially.
        if len(self.turns) == 0:
            return True
        return False

    def _validate_player_response(self, player: Player, utterance: str) -> bool:
        # Check responses for specific players
        if player == self.greeter:
            # Check rule: utterance starts with key word
            if not utterance.startswith("GREET:"):
                self.success = False
                return True
            # Check rule: required words are included
            utterance = utterance.lower()
            utterance = utterance.translate(str.maketrans("", "", string.punctuation))
            for required_word in self.required_words:
                if required_word not in utterance:
                    self.success = False
        return True

    def _on_after_turn(self, turn_idx: int):
        self.turns.append(self.success)

    def _after_add_player_response(self, player: Player, utterance: str):
        if player == self.greeter:
            self.add_user_message(self.greeted, utterance)

    def compute_scores(self) -> None:
        score = 0
        if self.success:
            score = 1
        self.log_episode_score('Accuracy', score)

# Players
Before actually initiating the **GameMaster**, it has to be defined what the players actually do. <br> 
One player is going to be the actual model **(the greeter)** and the other model is utilizing the generic **CustomResponseModel**, which is programmatic.

In [5]:
class Greeted(Player):

    def __init__(self, name):
        # In the SuperClass Player, there is a parameter self.model
        # The Player class accepts an instance of CustomResponseModel through its model parameter,
        # When super().__init__ is called, the constructor of the superclass is expecting a parameter (model)
        # That is where CustomResponseModel gets instantiated.
        super().__init__(CustomResponseModel())
        self.name = name

    def _custom_response(self, messages, turn_idx):
        return f"{self.name}: Hi, thanks for having me!"


class Greeter(Player):

    def __init__(self, model: Model):
        super().__init__(model)

    def _custom_response(self, messages, turn_idx):
        raise NotImplementedError("This should not be called, but the remote APIs.")

# NOTE: Look into the Player class in more detail

## Fitting to the Benchmark
- Now that there is a GameMaster and Player actions have been defined, the Game has to be fit into the benchmark.
- The following defines a standard way to get to the game master when the benchmark is called:

In [6]:
class HelloGameBenchmark(GameBenchmark):

    def __init__(self):
        # It gets a bit tricky here. This Class inherits from GameBenchmark, which inherits from GameResourceLocator
        # GameResourceLocator has a parameter 'name'. This is what is being constructed here
        # GameBenchmark calls its parent class to initialize the 'name' parameter
        super().__init__(GAME_NAME)

    def get_description(self):
        return "Hello game between a greeter and a greeted player"

    def create_game_master(self, experiment: Dict, player_models: List[Model]) -> GameMaster:
        return HelloGame(experiment, player_models)

In [7]:
hgb = HelloGameBenchmark()
hgm = hgb.create_game_master({"language": "en"}, [])
hgm.__dict__ # __dict__ attribute of the hgm object. - A dictionary storing all attributes of the Object.

{'name': 'hellogame',
 'logger': <Logger __main__ (INFO)>,
 'log_current_turn': -1,
 'interactions': {'players': {}, 'turns': []},
 'requests': [],
 'experiment': {'language': 'en'},
 'player_models': [],
 'players_by_names': OrderedDict(),
 'messages_by_names': {},
 'current_turn': 0,
 'language': 'en',
 'turns': [],
 'required_words': ['welcome', 'hello'],
 'success': True}

In [8]:
# NOTE: This will be merged into the object of the GameMaster class, see below.
this_experiment = {
      "name": "greet_en",
      "game_instances": [
        {
          "game_id": 0,
          "prompt": "Your task is to greet and happily welcome the other person with the name:\n\nPeter\n\nRules:\n\n1. You must start your message with 'GREET:'\n2. Your message must include 'Hello', 'welcome' and the other person's name\n\nImportant: You only have one try.\n\nLet's start.",
          "target_name": "Peter"
        } ],
      "language": "en"
    }

In [9]:
THIS_MODEL = 'gpt-4o-mini-2024-07-18'
llm = get_model_for(THIS_MODEL)
llm.set_gen_args(temperature = 0.0, max_tokens= 100) 

hgm = hgb.create_game_master(this_experiment, [llm])

In [10]:
hgm.__dict__

{'name': 'hellogame',
 'logger': <Logger __main__ (INFO)>,
 'log_current_turn': -1,
 'interactions': {'players': {}, 'turns': []},
 'requests': [],
 'experiment': {'name': 'greet_en',
  'game_instances': [{'game_id': 0,
    'prompt': "Your task is to greet and happily welcome the other person with the name:\n\nPeter\n\nRules:\n\n1. You must start your message with 'GREET:'\n2. Your message must include 'Hello', 'welcome' and the other person's name\n\nImportant: You only have one try.\n\nLet's start.",
    'target_name': 'Peter'}],
  'language': 'en'},
 'player_models': [gpt-4o-mini-2024-07-18],
 'players_by_names': OrderedDict(),
 'messages_by_names': {},
 'current_turn': 0,
 'language': 'en',
 'turns': [],
 'required_words': ['welcome', 'hello'],
 'success': True}

In [11]:
# The class DialogueGameMaster has a method 'setup' which, initializes the players by calling the on_setup method
hgm.setup(**this_experiment['game_instances'][0])
hgm.__dict__

{'name': 'hellogame',
 'logger': <Logger __main__ (INFO)>,
 'log_current_turn': -1,
 'interactions': {'players': OrderedDict([('GM', 'Game master for hellogame'),
               ('Player 1', 'Greeter, gpt-4o-mini-2024-07-18'),
               ('Player 2', 'Greeted, programmatic')]),
  'turns': []},
 'requests': [],
 'experiment': {'name': 'greet_en',
  'game_instances': [{'game_id': 0,
    'prompt': "Your task is to greet and happily welcome the other person with the name:\n\nPeter\n\nRules:\n\n1. You must start your message with 'GREET:'\n2. Your message must include 'Hello', 'welcome' and the other person's name\n\nImportant: You only have one try.\n\nLet's start.",
    'target_name': 'Peter'}],
  'language': 'en'},
 'player_models': [gpt-4o-mini-2024-07-18],
 'players_by_names': OrderedDict([('Player 1',
               <__main__.Greeter at 0x7f34e1a72320>),
              ('Player 2', <__main__.Greeted at 0x7f34e1a73340>)]),
 'messages_by_names': {'Player 1': [], 'Player 2': []},
 'cu

In [12]:
hgm.play()
hgm.__dict__


{'name': 'hellogame',
 'logger': <Logger __main__ (INFO)>,
 'log_current_turn': 0,
 'interactions': {'players': OrderedDict([('GM', 'Game master for hellogame'),
               ('Player 1', 'Greeter, gpt-4o-mini-2024-07-18'),
               ('Player 2', 'Greeted, programmatic')]),
  'turns': [[{'from': 'GM',
     'to': 'Player 1',
     'timestamp': '2024-08-30T23:26:25.202504',
     'action': {'type': 'send message',
      'content': "Your task is to greet and happily welcome the other person with the name:\n\nPeter\n\nRules:\n\n1. You must start your message with 'GREET:'\n2. Your message must include 'Hello', 'welcome' and the other person's name\n\nImportant: You only have one try.\n\nLet's start."}},
    {'from': 'Player 1',
     'to': 'GM',
     'timestamp': '2024-08-30T23:26:40.093168',
     'action': {'type': 'get message',
      'content': "GREET: Hello Peter, welcome! I'm so glad to have you here!"}},
    {'from': 'GM',
     'to': 'Player 2',
     'timestamp': '2024-08-30T23:2

### Results
In the end, we have a very detailed Dictionary, that includes all necessary data that you could need when the game was executed. <br>
**NOTE:** This definitely has to be studied a bit more detailed, but it already shows why it makes more sense to use the built-in tools to add your own games.

There are ways to inspect the results a bit more specific:

In [13]:
hgm.interactions['players']

OrderedDict([('GM', 'Game master for hellogame'),
             ('Player 1', 'Greeter, gpt-4o-mini-2024-07-18'),
             ('Player 2', 'Greeted, programmatic')])

In [14]:
hgm.interactions['turns']

[[{'from': 'GM',
   'to': 'Player 1',
   'timestamp': '2024-08-30T23:26:25.202504',
   'action': {'type': 'send message',
    'content': "Your task is to greet and happily welcome the other person with the name:\n\nPeter\n\nRules:\n\n1. You must start your message with 'GREET:'\n2. Your message must include 'Hello', 'welcome' and the other person's name\n\nImportant: You only have one try.\n\nLet's start."}},
  {'from': 'Player 1',
   'to': 'GM',
   'timestamp': '2024-08-30T23:26:40.093168',
   'action': {'type': 'get message',
    'content': "GREET: Hello Peter, welcome! I'm so glad to have you here!"}},
  {'from': 'GM',
   'to': 'Player 2',
   'timestamp': '2024-08-30T23:26:40.093550',
   'action': {'type': 'send message',
    'content': "GREET: Hello Peter, welcome! I'm so glad to have you here!"}},
  {'from': 'Player 2',
   'to': 'GM',
   'timestamp': '2024-08-30T23:26:40.093639',
   'action': {'type': 'get message',
    'content': 'Peter: Hi, thanks for having me!'}}]]

In [15]:
hgm.success

True

NOTE: Add an overall summarization here, as this could become quite important during the implementation of our game.